In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# Data Processing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,cross_validate,GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer

# Get variable names
# from varname import nameof

# Pipeline diagram
from sklearn import set_config

# MODELS
# from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, roc_auc_score

In [2]:
ausencias = pd.read_csv('./data/raw/Ausencias.csv')
definitivas = pd.read_csv('./data/raw/Definitivas_asignaturas.csv')
estudiantes = pd.read_csv('./data/raw/Listado_estudiantes.csv')
psat = pd.read_csv('./data/raw/Pruebas_PSAT.csv')
saber_11 = pd.read_csv('./data/raw/Pruebas_Saber_11.csv')
simulacro = pd.read_csv('./data/raw/Simulacro_pruebas_saber_11.csv')


In [3]:
saber_11 = saber_11.pivot_table(index=['codigo','anio_escolar'], columns=['asignatura'],values='resultado').reset_index()
saber_11 = saber_11.rename_axis(None, axis=1)
saber_11.columns = [f'{j.lower().replace("","ni").replace(" ","_").strip()}_saber_11'  for j in saber_11.columns ]
saber_11.rename(columns={f'codigo_saber_11':'codigo'}, inplace= True)
del saber_11['anio_escolar_saber_11']
del saber_11['global_saber_11']
saber_11_materias = [ j for j in saber_11.columns if j.endswith('saber_11') ]
for i in saber_11_materias:
    if i == 'ingles_saber_11':
        saber_11[i] = saber_11[i].apply(lambda x: 1 if x >=80 else 0)
    else:
        saber_11[i] = saber_11[i].apply(lambda x: 1 if x >=60 else 0)


In [4]:
psat = psat.pivot_table(index=['codigo','anio_escolar'], columns=['asignatura'],values='resultado').reset_index()
psat.drop_duplicates(subset= 'codigo', keep= 'first', inplace = True)
psat = psat.rename_axis(None, axis=1)
psat.columns = [f'{j.lower().replace(" ","_").strip()}_psat'  for j in psat.columns ]
psat.rename(columns={f'codigo_psat':'codigo'}, inplace= True)
del psat['anio_escolar_psat']
del psat['combinado_psat']

In [5]:
definitivas_10 = definitivas[definitivas.grado == 10]
definitivas_10 = definitivas_10.groupby(['codigo','asignatura'])['resultado'].mean().to_frame().reset_index()
definitivas_10 = definitivas_10.pivot_table(index=['codigo'], columns=['asignatura'],values='resultado').reset_index()
definitivas_10 = definitivas_10.rename_axis(None, axis=1)
definitivas_10.columns = [f'{j.lower().replace(" ","_").replace("","ni").strip()}_notas_10'  for j in definitivas_10.columns ]
definitivas_10.rename(columns={f'codigo_notas_10':'codigo'}, inplace= True)

definitivas_11 = definitivas[definitivas.grado == 11]
definitivas_11 = definitivas_11.groupby(['codigo','asignatura'])['resultado'].mean().to_frame().reset_index()
definitivas_11 = definitivas_11.pivot_table(index=['codigo'], columns=['asignatura'],values='resultado').reset_index()
definitivas_11 = definitivas_11.rename_axis(None, axis=1)
definitivas_11.columns = [f'{j.lower().replace(" ","_").replace("","ni").strip()}_notas_11'  for j in definitivas_11.columns ]
definitivas_11.rename(columns={f'codigo_notas_11':'codigo'}, inplace= True)

definitivas_12 = definitivas[definitivas.grado == 12]
definitivas_12 = definitivas_12.groupby(['codigo','asignatura'])['resultado'].mean().to_frame().reset_index()
definitivas_12 = definitivas_12.pivot_table(index=['codigo'], columns=['asignatura'],values='resultado').reset_index()
definitivas_12 = definitivas_12.rename_axis(None, axis=1)
definitivas_12.columns = [f'{j.lower().replace(" ","_").replace("","ni").strip()}_notas_12'  for j in definitivas_12.columns ]
definitivas_12.rename(columns={f'codigo_notas_12':'codigo'}, inplace= True)

In [6]:
print(simulacro.grado.value_counts())
simulacro['resultado'] = simulacro['resultado'].apply(lambda x: str(x).strip().replace(',','.'))
simulacro['resultado'] = simulacro['resultado'].apply(lambda x: float(x))


simulacro_11 = simulacro[simulacro.grado == 11]
simulacro_11 = simulacro_11.groupby(['codigo','asignatura'])['resultado'].mean().to_frame().reset_index()
simulacro_11 = simulacro_11.pivot_table(index=['codigo'], columns=['asignatura'],values='resultado').reset_index()
simulacro_11 = simulacro_11.rename_axis(None, axis=1)
simulacro_11.columns = [f'{j.lower().replace(" ","_").replace("","ni").strip()}_sim_11'  for j in simulacro_11.columns ]
simulacro_11.rename(columns={f'codigo_sim_11':'codigo'}, inplace= True)
del simulacro_11['def_sim_11']

simulacro_12 = simulacro[simulacro.grado == 12]
simulacro_12 = simulacro_12.groupby(['codigo','asignatura'])['resultado'].mean().to_frame().reset_index()
simulacro_12 = simulacro_12.pivot_table(index=['codigo'], columns=['asignatura'],values='resultado').reset_index()
simulacro_12 = simulacro_12.rename_axis(None, axis=1)
simulacro_12.columns = [f'{j.lower().replace(" ","_").replace("","ni").strip()}_sim_12'  for j in simulacro_12.columns ]
simulacro_12.rename(columns={f'codigo_sim_12':'codigo'}, inplace= True)
del simulacro_12['def_sim_12']

12    10829
11     6694
Name: grado, dtype: int64


In [7]:
df_10 = definitivas_10.merge(saber_11, on = 'codigo', how = 'inner')
df_10 = df_10.merge(psat, on = 'codigo', how = 'inner')

In [8]:
df_11 = definitivas_11.merge(saber_11, on = 'codigo', how = 'inner')
df_11 = df_11.merge(simulacro_11, on = 'codigo', how = 'inner')
del df_11['ciencias_sociales_notas_11']
df_11.dropna(inplace=True)

In [9]:
df_12 = definitivas_12.merge(saber_11, on = 'codigo', how = 'inner')
df_12 = df_12.merge(simulacro_12, on = 'codigo', how = 'inner')
df_12.dropna(inplace=True)

In [10]:
def keep_materia_saber(df:pd.DataFrame,materia:str)->pd.DataFrame:
    """Get Rid all the saber 11 values, and just keep the "materia" result of the parameter

    Args:
        df (pd.DataFrame): Dataframe with all the columns (all saber 11 scores)
        materia (str): the name of the target

    Returns:
        (pd.DataFrame): the dataframe transformed with target value corresponding to the "materia" saber 11
    """
    col_add = df[[materia]].copy()
    columns = [ i for i in df.columns if not i.endswith('saber_11') ]
    df = df[columns]
    df[materia] = col_add

    return df

math_column = 'matematicas_saber_11'
lectura_column = 'lectura_critica_saber_11'
ingles_column = 'ingles_saber_11'
ciencias_column = 'ciencias_saber_11'
sociales_column = 'sociales_y_ciudadanas_saber_11'

In [11]:
# Matematicas

df_10_MATH = keep_materia_saber(df_10,math_column)
df_11_MATH = keep_materia_saber(df_11,math_column)
df_12_MATH = keep_materia_saber(df_12,math_column)

In [12]:
# lectura
df_10_LECT = keep_materia_saber(df_10,lectura_column)
df_11_LECT = keep_materia_saber(df_11,lectura_column)
df_12_LECT = keep_materia_saber(df_12,lectura_column)

In [13]:
# Ingles
df_10_INGLES= keep_materia_saber(df_10,ingles_column)
df_11_INGLES= keep_materia_saber(df_11,ingles_column)
df_12_INGLES= keep_materia_saber(df_12,ingles_column)


In [14]:
# ciencias_column
df_10_CIENCIAS = keep_materia_saber(df_10,ciencias_column)
df_11_CIENCIAS = keep_materia_saber(df_11,ciencias_column)
df_12_CIENCIAS = keep_materia_saber(df_12,ciencias_column)

In [15]:
# sociales_column
df_10_SOCIALES = keep_materia_saber(df_10,sociales_column)
df_11_SOCIALES = keep_materia_saber(df_11,sociales_column)
df_12_SOCIALES = keep_materia_saber(df_12,sociales_column)

In [16]:
df_12_SOCIALES.columns

Index(['codigo', 'ciencias_sociales_notas_12', 'disciplina_notas_12',
       'economia_notas_12', 'espaniol_notas_12', 'estadistica_notas_12',
       'filosofia_notas_12', 'fisica_notas_12', 'ingles_notas_12',
       'matematicas_notas_12', 'quimica_notas_12', 'biologia_sim_12',
       'cts_sim_12', 'competencias_ciudadanas_sim_12', 'fisica_sim_12',
       'ingles_sim_12', 'lectura_critica_sim_12', 'matematicas_(espec)_sim_12',
       'matematicas_(cuant)_sim_12', 'quimica_sim_12', 'sociales_sim_12',
       'sociales_y_ciudadanas_saber_11'],
      dtype='object')

In [17]:
def evaluation(y_test, y_predict, title = 'Confusion Matrix'):
    cm = confusion_matrix(y_test, y_predict)
    precision = precision_score(y_test, y_predict)
    recall = recall_score(y_test, y_predict)
    accuracy = accuracy_score(y_test,y_predict)
    f1 = f1_score(y_test,y_predict)
    roc = roc_auc_score(y_test, y_predict)
    metrics = {'Accuracy':accuracy,
                'precision':precision,
                'Recall':recall,
                'f1':f1,
                'roc' : roc
                 }

    metrics_df = pd.DataFrame([metrics])
    # print('Recall: ', recall)
    # print('Accuracy: ', accuracy)
    # print('Precision: ', precision)
    # print('F1: ', f1)
    display(metrics_df)
    sns.heatmap(cm,  cmap= 'Blues', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)
    
    plt.show();
    return metrics_df

In [22]:
dataframes_dict = {
    'df_10_MATH' : 'df_10_MATH',
    'df_10_LECT' : 'df_10_LECT',
    'df_10_INGLES' : 'df_10_INGLES',
    'df_10_CIENCIAS' : 'df_10_CIENCIAS',
    'df_10_SOCIALES' : 'df_10_SOCIALES',
    'df_11_MATH' : 'df_11_MATH',
    'df_11_LECT' : 'df_11_LECT',
    'df_11_INGLES' : 'df_11_INGLES',
    'df_11_CIENCIAS' : 'df_11_CIENCIAS',
    'df_11_SOCIALES' : 'df_11_SOCIALES',
    'df_12_MATH' : 'df_12_MATH',
    'df_12_LECT' : 'df_12_LECT',
    'df_12_INGLES' : 'df_12_INGLES',
    'df_12_CIENCIAS' : 'df_12_CIENCIAS',
    'df_12_SOCIALES' : 'df_12_SOCIALES',
    }

dataframes = [df_10_MATH,df_10_LECT,df_10_INGLES,df_10_CIENCIAS,df_10_SOCIALES,
df_11_MATH,df_11_LECT,df_11_INGLES,df_11_CIENCIAS,df_11_SOCIALES,
df_12_MATH,df_12_LECT,df_12_INGLES,df_12_CIENCIAS,df_12_SOCIALES]


In [ ]:

datasets_models_dict = {
    'df_10_MATH': 'knn_classifier',
    'df_10_LECT': 'knn_classifier',
    'df_10_INGLES': 'knn_classifier',
    'df_10_CIENCIAS' : 'gb_classifier',
    'df_10_SOCIALES' : 'gb_classifier',
    'df_11_MATH': 'knn_classifier',
    'df_11_LECT' : 'rf_classifier',
    'df_11_INGLES' : 'rf_classifier',
    'df_11_CIENCIAS' : 'gb_classifier',
    'df_11_SOCIALES': 'knn_classifier',
    'df_12_MATH' : 'rf_classifier',
    'df_12_LECT' : 'rf_classifier',
    'df_12_INGLES' : 'rf_classifier',
    'df_12_CIENCIAS' : 'rf_classifier',
    'df_12_SOCIALES' : 'gb_classifier',}

contador = 0
logs_metrics = pd.DataFrame()
for i in dataframes:
    del i['codigo']
    X = i.drop(columns = i.columns[-1], axis = 1)
    y = i[f'{i.columns[-1]}']
    X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


    model_choose = datasets_models_dict[list(datasets_models_dict.keys())[contador]]
    dataset_name = list(datasets_models_dict.keys())[contador]
    if model_choose == 'knn_classifier':

        model = KNeighborsClassifier()


    elif model_choose == 'rf_classifier':

        model = RandomForestClassifier()
 
      
    elif model_choose == 'gb_classifier':

        model = GradientBoostingClassifier()
        

    # Build the pipeline
    
   
    # Build the pipeline
    pipeline = Pipeline([
                    # ('Smote', SMOTE(random_state=0)),
                    ('Model', model)
                ])
    
    pipeline.fit(X_train, y_train);
    y_predict = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:,1]
    print(list(datasets_models_dict.keys())[contador])
    logs_metrics_ = evaluation(y_test, y_predict)
    logs_metrics_['data_Set'] = list(datasets_models_dict.keys())[contador]
    logs_metrics = logs_metrics.append(logs_metrics_)
    dataset_name_joblib = dataset_name.replace('df_','')
    joblib.dump(pipeline,f'./models/{dataset_name_joblib}_{model_choose}.joblib')
    print('-'*50)
    contador +=1


In [25]:
# FINALIZE DE MODELS

datasets_models_dict = {
    'df_10_MATH': 'knn_classifier',
    'df_10_LECT': 'knn_classifier',
    'df_10_INGLES': 'knn_classifier',
    'df_10_CIENCIAS' : 'gb_classifier',
    'df_10_SOCIALES' : 'gb_classifier',
    'df_11_MATH': 'knn_classifier',
    'df_11_LECT' : 'rf_classifier',
    'df_11_INGLES' : 'rf_classifier',
    'df_11_CIENCIAS' : 'gb_classifier',
    'df_11_SOCIALES': 'knn_classifier',
    'df_12_MATH' : 'rf_classifier',
    'df_12_LECT' : 'rf_classifier',
    'df_12_INGLES' : 'rf_classifier',
    'df_12_CIENCIAS' : 'rf_classifier',
    'df_12_SOCIALES' : 'gb_classifier',}

contador = 0
logs_metrics = pd.DataFrame()
for i in dataframes:
    try:
        del i['codigo']
    except:
        pass
    X = i.drop(columns = i.columns[-1], axis = 1)
    y = i[f'{i.columns[-1]}']
    # X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)


    model_choose = datasets_models_dict[list(datasets_models_dict.keys())[contador]]
    dataset_name = list(datasets_models_dict.keys())[contador]
    if model_choose == 'knn_classifier':

        model = KNeighborsClassifier()


    elif model_choose == 'rf_classifier':

        model = RandomForestClassifier()
 
      
    elif model_choose == 'gb_classifier':

        model = GradientBoostingClassifier()
        

    # Build the pipeline
    
   
    # Build the pipeline
    pipeline = Pipeline([
                    # ('Smote', SMOTE(random_state=0)),
                    ('Model', model)
                ])
    
    pipeline.fit(X, y);
    # y_predict = pipeline.predict(X_test)
    # y_pred_proba = pipeline.predict_proba(X_test)[:,1]
    # print(list(datasets_models_dict.keys())[contador])
    # logs_metrics_ = evaluation(y_test, y_predict)
    # logs_metrics_['data_Set'] = list(datasets_models_dict.keys())[contador]
    # logs_metrics = logs_metrics.append(logs_metrics_)
    dataset_name_joblib = dataset_name.replace('df_','')
    joblib.dump(pipeline,f'./models/{dataset_name_joblib}_{model_choose}.joblib')
    print('-'*50)
    contador +=1


--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
